In [ ]:
import os
import pandas as pd
from micom.workflows import build
import cobra
import micom
from micom.media import minimal_medium
import matplotlib.pyplot as plt
import random

In [ ]:
def xml_dict(directory_path, abundance = 1, extension='.xml'):
    # List all files in the directory with the specified extension
    files = [file for file in os.listdir(directory_path) if file.endswith(extension)]

    # Create a DataFrame to hold the data
    data = {
        "id": [],
        "file": [],
        "abundance": [],
        "sample_id": []
    }

    # Populate the DataFrame with information from the files
    for xml in files:
        filename = os.path.splitext(xml)[0]  # Get the filename without extension
        abundance = int(random.randint(1, 100))
        data["id"].append(filename)
        data["file"].append(os.path.join(directory_path, xml))
        data["abundance"].append(abundance)  # Assuming equal abundance for all
        data["sample_id"].append("sample1")  # Assuming a single sample

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)

    return df


In [ ]:
directory_path = '/Users/edwin/Downloads/reconstructions/dfba/'  

In [ ]:
df = xml_dict(directory_path)
print(df)

In [ ]:
manifest = build(df, out_folder=directory_path, model_db=None, threads=2)

In [ ]:
from micom.data import test_db
from micom.workflows import build
from micom.data import test_data

data = test_data()
data

manifest = build(data, out_folder="models", model_db=test_db, cutoff=0.0001, threads=2)

In [ ]:
 
def xml_dict(directory_path, extension='.xml'):
    # List all files in the directory with the specified extension
    files = [file for file in os.listdir(directory_path) if file.endswith(extension)]

    # Create a DataFrame to hold the data
    data = {
        "id": [],
        "genus": [],
        "species": [],
        "strain": [],
        "reactions": [],
        "metabolites": [],
        "file": [],
        "abundance": [],
        "sample_id": []
    }

    # Populate the DataFrame with information from the files
    for xml in files:
        filename = os.path.splitext(xml)[0]  # Get the filename without extension
        parts = filename.split('_')
        abundance = int(random.randint(1, 100))

        # Extract genus, species, and strain
        genus = parts[0]
        species = parts[1]
        strain = '_'.join(parts[2:])  # Combine all remaining parts for the strain

        # Construct the full path to the file
        full_path = os.path.join(directory_path, xml)
        
        # Read the model using cobra
        model = cobra.io.read_sbml_model(full_path)

        data["id"].append(filename)
        data["genus"].append(genus)
        data["species"].append(species)
        data["strain"].append(strain)
        data["reactions"].append(len(model.reactions))
        data["metabolites"].append(len(model.metabolites))
        data["file"].append(full_path)
        data["abundance"].append(abundance)  # Assuming equal abundance for all
        data["sample_id"].append("sample1")  # Assuming a single sample


    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)

    return df

In [ ]:
df = xml_dict(directory_path)

# Display the DataFrame
print(df)

In [ ]:
from micom import Community

com = Community(df)
print("Build a community with a total of {} reactions.".format(len(com.reactions)))

In [ ]:
com.medium["EX_fru_m"]

In [ ]:
sol = com.cooperative_tradeoff(fraction=0.5, fluxes= True)

In [ ]:
sol

In [ ]:
fluxes = sol.fluxes["EX_glc_D(e)"]
fluxes

In [ ]:
med = micom.media.minimal_medium(com, 0.8, min_growth=0.8)

In [ ]:
med["EX_glc_D_m"] = 1000
med["EX_fru_m"] = 1000

for key in med.keys():
    if "MG"in key:
        med[key] = 10
    else:
        med[key] = 1000

In [ ]:
com.medium['EX_glc_D_m']

In [ ]:
com.medium = med
com.optimize()

In [ ]:
solution = com.cooperative_tradeoff(fraction=0.5, fluxes= True)

In [ ]:
solution.fluxes['EX_glc_D_m'].medium

In [ ]:
so = com.optimize()
so.objective_value

In [ ]:
for key in com.reactions:
    if "biomass" in key.id:
        print(key.id)

In [ ]:
for i in so.members.index:
    if i != "medium":
        growth_rate = sol.members.growth_rate[i]
        print(growth_rate)

In [ ]:
df = pd.DataFrame({
    'reaction': med.index,
    'flux': [med[i] for i in med.index]
})

In [ ]:
df

In [ ]:
def dfba_timestep(
        model,
        current_state,
        kinetic_params,
        species_growth,
        substrate_update_reactions,
        dt,
        biomass_identifier='biomass'
):
    updated_state = current_state.copy()
    
    for substrate, reaction_id in substrate_update_reactions.items():
        Km, Vmax = kinetic_params[substrate]
        substrate_concentration = updated_state[substrate]
        uptake_rate = Vmax * substrate_concentration / (Km + substrate_concentration)
        model.medium[reaction_id] = uptake_rate
        print(uptake_rate)

    solution = model.optimize()
    sol = model.cooperative_tradeoff(fraction=0.5, fluxes= True)
    
    if solution.status == 'optimal':
        current_biomass = current_state[biomass_identifier]
        biomass_growth_rate = solution.objective_value
        updated_state[biomass_identifier] += biomass_growth_rate * current_biomass * dt
        
        for key in species_growth.keys():
            growth = species_growth[key]
            species_growth_rate = sol.members.growth_rate[key]
            updated_state[key] += species_growth_rate * growth * dt
            
        for substrate, reaction_id in substrate_update_reactions.items():
            flux = sol.fluxes[reaction_id].medium
            updated_state[substrate] = max(updated_state[substrate] + flux * current_biomass * dt, 0)
    else:
        # Handle non-optimal solutions if necessary
        pass

    return updated_state


def perform_dfba(
        model,
        initial_conditions,
        kinetic_params,
        species_growth,
        time_points,
        substrate_update_reactions,
        dt,
        biomass_identifier='biomass'
):
    results = {key: [value] for key, value in initial_conditions.items()}
    current_state = initial_conditions

    for t_i in range(1, len(time_points)):
        current_state = dfba_timestep(model,
                                      current_state,
                                      kinetic_params,
                                      species_growth,
                                      substrate_update_reactions,
                                      dt,
                                      biomass_identifier)

        # Store the current state in results
        for key in current_state:
            if key in results:
                results[key].append(current_state[key])
            else:
                results[key] = [current_state[key]]

    return results

def plot_dfba_results(time_points, dfba_results, title=''):
    plt.figure(figsize=(8, 5))
    for key, values in dfba_results.items():
        plt.plot(time_points, values, label=key.capitalize())

    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.legend()
    plt.show()

In [ ]:
# Define initial conditions, kinetic parameters, and other necessary inputs
species_growth = {}
for i in so.members.index:
    if i != "medium":
        species_growth[i] = sol.members.growth_rate[i]


initial_conditions = {
    'biomass': so.objective_value,  # Initial biomass concentration
    'glucose': 100, # Initial glucose concentration
    'fructose': 100   # Initial acetate concentration
}

initial_conditions_updated = {**initial_conditions, **species_growth} # combine the species growth and initial conditions


kinetic_params = {
    'glucose': (0.5, 2), # Km and Vmax for glucose
    'fructose': (5, 2)  # Km and Vmax for acetate
}
substrate_update_reactions = {
    'glucose': 'EX_glc_D_m',  # Exchange reaction ID for glucose
    'fructose': 'EX_fru_m'       # Exchange reaction ID for acetate
}

# simulation conditions
t_n = 100 # number of time points
dt = 0.0001 # Time step, matching your setup
time_points = list(range(0, t_n))  # Simulation time points, matching your setup


In [ ]:
initial_conditions_updated

In [ ]:
# Make sure to adjust the perform_dfba function if needed to initialize result arrays with sufficient size
dfba_results = perform_dfba(
    com, 
    initial_conditions_updated, 
    kinetic_params, 
    species_growth,
    time_points,  
    substrate_update_reactions, 
    dt,
    'biomass'
)

In [ ]:
dfba_results

In [ ]:
nutrient_results = {key: value for key, value in dfba_results.items() if key != 'biomass'}
biomass_results = {key: value for key, value in dfba_results.items() if key == 'biomass'}
plot_dfba_results(time_points, nutrient_results)
plot_dfba_results(time_points, biomass_results)

In [ ]:
plot_dfba_results(time_points, dfba_results)